In [1]:
import requests
import json
import pandas as pd
import numpy as np
import ift6758
from pathlib import Path

In [2]:
import os
from pathlib import Path
import logging
from flask import Flask, jsonify, request, abort, g
import xgboost
import requests
import pandas as pd
from comet_ml import API
import joblib

import sklearn
import ift6758

In [3]:
try:
    hey = requests.get("http://0.0.0.0:5000/hello")
except Exception as e:
    print(e)

In [4]:
hey.text

'Hello!\n\nThe purpose of this page is to know if requests work.'

In [5]:
logs = requests.get("http://0.0.0.0:5000/logs")
logs.json()[-5:]

['2022-12-21 21:55:41,748;INFO;Before first request - Start',
 "2022-12-21 21:55:42,321;INFO;Default model properly loaded: {'workspace': 'ift-6758-projet-quipe-13', 'model': 'xgboost-best-all-features', 'version': '1.0.0', 'filename': 'XGBoost_best_all_features.pkl'}.",
 '2022-12-21 21:55:42,321;INFO;Before first request - End',
 '2022-12-21 21:55:42,322;INFO;Accessed page /hello - TESTING',
 '2022-12-21 21:55:42,349;INFO;Accessed page /logs']

In [6]:
default_model = {
    'workspace': 'ift-6758-projet-quipe-13',
    'model': 'xgboost-base-all-features',
    'version': '1.0.0'
}

In [7]:
model = requests.post("http://0.0.0.0:5000/download_registry_model", json=default_model)

In [8]:
model.text

'"Updated model from CometML api and downloaded locally."\n'

In [9]:
model

<Response [200]>

In [10]:
df = pd.DataFrame(np.random.randn(5266, 29))

In [11]:
X_dict = {}
X_values = df.values.tolist()
X_dict['values'] = X_values

In [12]:
list(X_dict.keys())[0]

'values'

In [13]:
pred = requests.post("http://0.0.0.0:5000/predict", json=X_dict)

In [14]:
pred.json()[:10]

[0.5731564164161682,
 0.4883981943130493,
 0.2846738398075104,
 0.1125500351190567,
 0.6350976228713989,
 0.21481068432331085,
 0.2358998954296112,
 0.38799890875816345,
 0.629584550857544,
 0.5653513073921204]

In [15]:
df = pd.DataFrame(np.random.randn(5266, 27))

In [16]:
X_dict = {}
X_values = df.values.tolist()
X_dict['values'] = X_values

In [17]:
pred = requests.post("http://0.0.0.0:5000/predict", json=X_dict)

In [19]:
pred.json()

'Feature shape mismatch, expected: 29, got 27'

In [20]:
df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1.234908,-0.734066,-1.253741,-1.093556,1.376187,-0.878768,-0.405272,-0.569790,0.212552,-0.214741,...,0.198800,-1.547909,-0.806561,-0.549489,1.246926,0.366559,0.023231,-1.170599,0.434747,0.489296
1,-0.592501,-0.859050,1.973289,-0.440976,0.011455,-1.450020,-0.674640,-0.435448,1.243778,0.360943,...,0.465301,0.539635,0.625122,1.126807,-0.542630,0.740207,1.478395,-0.423283,-0.016349,0.772436
2,-0.660579,0.233835,-0.134521,0.088873,-1.536675,-1.846730,-1.184343,0.542409,1.369142,-0.564131,...,-0.448345,0.377150,-0.296248,1.406879,1.843037,-1.765820,-1.679669,-0.089361,-1.463714,1.246184
3,1.461116,-0.638603,0.783887,1.312160,-0.197346,-0.701314,-0.567216,-0.232465,0.577321,-1.432896,...,-0.783781,0.684234,-0.076710,-0.930033,0.138152,0.302081,0.306951,-0.318013,0.579737,0.182124
4,1.574554,-0.662837,0.743875,-1.392540,0.969636,-2.548336,-0.968892,0.578839,0.124201,0.055338,...,0.409174,0.159819,-0.806604,-0.985790,0.076453,1.136735,-2.114656,-0.061645,-0.888104,-0.656723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5261,-0.616106,-0.336303,0.393357,-0.233537,0.314431,2.766134,0.759143,0.836599,0.536985,-2.821661,...,0.774616,-0.689187,0.187204,0.414936,0.355548,-0.585850,0.145940,0.985620,1.356129,-0.220339
5262,-0.978820,-0.724097,-0.861911,-0.442103,-0.099027,0.494181,-0.305045,-0.108759,1.937575,-0.445053,...,-0.574153,0.794665,0.426413,0.543677,-2.685734,0.929881,0.375776,-0.891258,0.829005,-1.748480
5263,-1.511991,1.209361,0.205429,0.514973,-1.232118,0.459006,1.040373,-0.313490,0.482681,-0.689133,...,-2.187838,1.110941,0.372466,0.794642,-2.000957,-0.387718,-0.276130,-0.315218,0.978121,1.301628
5264,-1.794219,0.713019,-0.338968,-0.712401,-0.890946,-1.320608,0.498217,-0.823092,0.662843,1.605881,...,0.182033,-0.475008,-1.144971,0.928936,-1.884905,0.582951,-2.897831,-0.524965,1.495949,-0.604869


In [24]:
if xd.values is None:
    print('hello')